LeNet5, AlexNet, VGG, ResNet18, SENet18 and GoogleNet Architrctures are defined

**LeNet5 ARCHITECTURE**

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

**AlexNet ARCHITECTURE**

In [2]:
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=10, input_channels=3):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 1 * 1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

**VGG ARCHITECTURE**

In [3]:
import torch.nn as nn

class VGG(nn.Module):
    def __init__(self, num_classes=10, input_channels=3):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


**ResNet18 ARCHITECTURE**

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, num_classes=10, input_channels=3):
        super(ResNet18, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride):
        strides = [stride] + [1]*(blocks-1)
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.max_pool2d(out, kernel_size=3, stride=2, padding=1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, (1,1))
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

**SENet18 ARCHITECTURE**

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(in_channels, in_channels // reduction, bias=False)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels, bias=False)

    def forward(self, x):
        b, c, _, _ = x.size()
        y = F.adaptive_avg_pool2d(x, 1).view(b, c)
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y)).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SEBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, reduction=16):
        super(SEBasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.se = SEBlock(planes, reduction)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.se(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class SENet18(nn.Module):
    def __init__(self, num_classes=10, input_channels=3):
        super(SENet18, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(SEBasicBlock, 64, 2, stride=1)
        self.layer2 = self._make_layer(SEBasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(SEBasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(SEBasicBlock, 512, 2, stride=2)
        self.linear = nn.Linear(512 * SEBasicBlock.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride):
        strides = [stride] + [1] * (blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.max_pool2d(out, kernel_size=3, stride=2, padding=1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

**GoogleNet ARCHITECTURE**

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()
        self.branch1 = nn.Conv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = branch1, branch2, branch3, branch4
        return torch.cat(outputs, dim=1)

class GoogleNet(nn.Module):
    def __init__(self, num_classes=10, input_channels=3, aux_logits=True):
        super(GoogleNet, self).__init__()
        self.aux_logits = aux_logits
        self.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1)
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        if self.aux_logits and self.training:
            aux1 = self.aux1(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if self.aux_logits and self.training:
            aux2 = self.aux2(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)

        if self.aux_logits and self.training:
            return torch.cat((aux1, aux2, x), dim=1)
        return x

class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.conv = nn.Conv2d(in_channels, 128, kernel_size=1)
        self.fc1 = nn.Linear(128*4*4, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.7)

    def forward(self, x):
        x = F.adaptive_avg_pool2d(x, (4, 4))
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x), inplace=True)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

**DOWNLOADING DATA**

In [7]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms

def get_datasets():
    transform_mnist = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    transform_cifar = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
    mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

    fmnist_train = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform_mnist)
    fmnist_test = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform_mnist)

    cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
    cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

    return {
        'MNIST': (mnist_train, mnist_test),
        'FMNIST': (fmnist_train, fmnist_test),
        'CIFAR-10': (cifar10_train, cifar10_test)
    }


In [8]:
#Function call for getting the data
data = get_datasets()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 4970912.88it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 159303.97it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 25522786.04it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 170498071/170498071 [00:10<00:00, 16246578.17it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


**USER DEFINED FUNCTION FOR TRAINING AND EVALUATING THE MODEL**

In [10]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import precision_score, recall_score, f1_score

def training_and_evaluating(model, train_loader, test_loader, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

  train_loss_list = []

  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    
    #getting the loss curve
    train_loss = running_loss / len(train_loader)
    train_loss_list.append(train_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

    # Test the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    #calculating accuracy
    accuracy = 100 * correct / total

  #other evaluation metrics
  precision = precision_score(labels.cpu(), predicted.cpu(),average='macro')
  recall = recall_score(labels.cpu(), predicted.cpu(),average='macro')
  f1 = f1_score(labels.cpu(), predicted.cpu(),average='macro')

  print(f'\nAccuracy: {accuracy:.2f}%')
  print(f'Precision: {precision:.2f}')
  print(f'Recall: {recall:.2f}')
  print(f'F1-score: {f1:.2f}')

  metrics = {'Accuracy':accuracy, 'Precision':precision, 'Recall':recall, 'F1-score':f1 }

  return train_loss_list, metrics

**MNIST DATA**

In [ ]:
#creating a list for evaluation metrics of all models for MNIST Dataset
metrics_m = []

In [ ]:
#Data_loader for MNIST
from torch.utils.data import DataLoader
train_loader = DataLoader(data['MNIST'][0], batch_size=64, shuffle=True)
test_loader = DataLoader(data['MNIST'][1], batch_size=1000, shuffle=False)

**Function call for each model**

In [ ]:
#parameters of training_and_evaluating fumction --> model, train_loader, test_loader, num_epochs

model_1 = LeNet5(num_classes=10).to(device)
m_train_loss_list_1, m_metrics_1 = training_and_evaluating(model_1, train_loader, test_loader, 10)
m_metrics_1['Dataset']='MNIST'
m_metrics_1['Model name']='LeNet5'
metrics_m.append(m_metrics_1)

Epoch [1/10], Loss: 0.2037
Epoch [2/10], Loss: 0.0610
Epoch [3/10], Loss: 0.0446
Epoch [4/10], Loss: 0.0362
Epoch [5/10], Loss: 0.0295
Epoch [6/10], Loss: 0.0239
Epoch [7/10], Loss: 0.0220
Epoch [8/10], Loss: 0.0187
Epoch [9/10], Loss: 0.0156
Epoch [10/10], Loss: 0.0144

Accuracy: 98.94%
Precision: 0.99
Recall: 0.99
F1-score: 0.99


In [ ]:
model_2 = AlexNet(num_classes=10).to(device)
m_train_loss_list_2, m_metrics_2 = training_and_evaluating(model_2, train_loader, test_loader, 10)
m_metrics_2['Dataset']='MNIST'
m_metrics_2['Model name']='AlexNet'
metrics_m.append(m_metrics_2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/10], Loss: 0.3693
Epoch [2/10], Loss: 0.1263
Epoch [3/10], Loss: 0.1050
Epoch [4/10], Loss: 0.0920
Epoch [5/10], Loss: 0.0753
Epoch [6/10], Loss: 0.0585
Epoch [7/10], Loss: 0.0726
Epoch [8/10], Loss: 0.0682
Epoch [9/10], Loss: 0.0575
Epoch [10/10], Loss: 0.0459

Accuracy: 98.22%
Precision: 0.99
Recall: 0.99
F1-score: 0.99


In [ ]:
model_3 = VGG(num_classes=10).to(device)
m_train_loss_list_3, m_metrics_3 = training_and_evaluating(model_3, train_loader, test_loader, 10)
m_metrics_3['Dataset']='MNIST'
m_metrics_3['Model name']='VGG'
metrics_m.append(m_metrics_3)

Epoch [1/10], Loss: 0.8652
Epoch [2/10], Loss: 0.1503
Epoch [3/10], Loss: 0.1338
Epoch [4/10], Loss: 0.0840
Epoch [5/10], Loss: 0.0774
Epoch [6/10], Loss: 0.0792
Epoch [7/10], Loss: 0.0743
Epoch [8/10], Loss: 0.0607
Epoch [9/10], Loss: 0.0578
Epoch [10/10], Loss: 0.0489

Accuracy: 98.53%
Precision: 0.99
Recall: 0.98
F1-score: 0.99


In [ ]:
model_4 = ResNet18(num_classes=10).to(device)
m_train_loss_list_4, m_metrics_4 = training_and_evaluating(model_4, train_loader, test_loader, 10)
m_metrics_4['Dataset'] ='MNIST'
m_metrics_4['Model name'] = 'ResNet18'
metrics_m.append(m_metrics_4)

Epoch [1/10], Loss: 0.1330
Epoch [2/10], Loss: 0.0605
Epoch [3/10], Loss: 0.0464
Epoch [4/10], Loss: 0.0393
Epoch [5/10], Loss: 0.0356
Epoch [6/10], Loss: 0.0301
Epoch [7/10], Loss: 0.0285
Epoch [8/10], Loss: 0.0230
Epoch [9/10], Loss: 0.0243
Epoch [10/10], Loss: 0.0206

Accuracy: 99.43%
Precision: 0.99
Recall: 0.99
F1-score: 0.99


In [ ]:
model_5 = SENet18(num_classes=10).to(device)
m_train_loss_list_5, m_metrics_5 = training_and_evaluating(model_5, train_loader, test_loader, 10)
m_metrics_5['Dataset']='MNIST'
m_metrics_5['Model name']='SENet18'
metrics_m.append(m_metrics_5)

Epoch [1/10], Loss: 0.1169
Epoch [2/10], Loss: 0.0530
Epoch [3/10], Loss: 0.0407
Epoch [4/10], Loss: 0.0350
Epoch [5/10], Loss: 0.0324
Epoch [6/10], Loss: 0.0264
Epoch [7/10], Loss: 0.0239
Epoch [8/10], Loss: 0.0212
Epoch [9/10], Loss: 0.0191
Epoch [10/10], Loss: 0.0158

Accuracy: 99.16%
Precision: 0.99
Recall: 0.99
F1-score: 0.99


In [ ]:
model_6 = GoogleNet(num_classes=10).to(device)
m_train_loss_list_6, m_metrics_6 = training_and_evaluating(model_6, train_loader, test_loader, 10)
m_metrics_6['Dataset']='MNIST'
m_metrics_6['Model name']='GoogleNet'
metrics_m.append(m_metrics_6)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/10], Loss: 0.3853
Epoch [2/10], Loss: 0.0956
Epoch [3/10], Loss: 13.7884
Epoch [4/10], Loss: 2.3013
Epoch [5/10], Loss: 2.3013
Epoch [6/10], Loss: 2.3013
Epoch [7/10], Loss: 2.3013
Epoch [8/10], Loss: 2.3013
Epoch [9/10], Loss: 2.3013
Epoch [10/10], Loss: 2.3013

Accuracy: 9.80%
Precision: 0.01
Recall: 0.10
F1-score: 0.02


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Creating CSV files**

Creating a loss dictionary for every dataset and converting them to csv files, so that we can download the csv file for particular dataset before runtime is exhausted and use it for plotting.

The evaluation metrics are also loaded in a csv file.

In [ ]:
import pandas as pd
m_loss = {}
m_loss['LeNet5'] = m_train_loss_list_1
m_loss['AlexNet'] = m_train_loss_list_2
m_loss['VGG'] = m_train_loss_list_3
m_loss['ResNet18'] = m_train_loss_list_4
m_loss['SENet18'] = m_train_loss_list_5
m_loss['GoogleNet'] = m_train_loss_list_6

df_m_loss = pd.DataFrame(m_loss)
df_m_loss.to_csv("mnist_loss.csv",index=False)


In [ ]:
df_m_metrics = pd.DataFrame(metrics_m)
df_m_metrics.to_csv("mnist_metrics.csv",index=False)

**FASHION MNIST DATA**

In [ ]:
#creating a list for evaluation metrics of all models for Fashion MNIST Dataset
metrics_f = []

In [ ]:
#Data_loader for FASHION MNIST
from torch.utils.data import DataLoader
train_loader = DataLoader(data['FMNIST'][0], batch_size=64, shuffle=True)
test_loader = DataLoader(data['FMNIST'][1], batch_size=1000, shuffle=False)

**Function call for each model**

In [ ]:
#parameters of training_and_evaluating fumction --> model, train_loader, test_loader, num_epochs

model_1 = LeNet5(num_classes=10).to(device)
f_train_loss_list_1, f_metrics_1 = training_and_evaluating(model_1, train_loader, test_loader, 10)
f_metrics_1['Dataset']='FASHION MNIST'
f_metrics_1['Model name']='LeNet5'
metrics_f.append(f_metrics_1)

Epoch [1/10], Loss: 0.5395
Epoch [2/10], Loss: 0.3505
Epoch [3/10], Loss: 0.3092
Epoch [4/10], Loss: 0.2838
Epoch [5/10], Loss: 0.2622
Epoch [6/10], Loss: 0.2453
Epoch [7/10], Loss: 0.2318
Epoch [8/10], Loss: 0.2200
Epoch [9/10], Loss: 0.2059
Epoch [10/10], Loss: 0.1956

Accuracy: 90.43%
Precision: 0.91
Recall: 0.91
F1-score: 0.91


In [ ]:
model_2 = AlexNet(num_classes=10).to(device)
f_train_loss_list_2, f_metrics_2 = training_and_evaluating(model_2, train_loader, test_loader, 10)
f_metrics_2['Dataset']='FASHION MNIST'
f_metrics_2['Model name']='AlexNet'
metrics_f.append(f_metrics_2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/10], Loss: 0.6522
Epoch [2/10], Loss: 0.4300
Epoch [3/10], Loss: 0.3843
Epoch [4/10], Loss: 0.3579
Epoch [5/10], Loss: 0.3475
Epoch [6/10], Loss: 0.3169
Epoch [7/10], Loss: 0.3087
Epoch [8/10], Loss: 0.2907
Epoch [9/10], Loss: 0.2840
Epoch [10/10], Loss: 0.2683

Accuracy: 89.14%
Precision: 0.90
Recall: 0.91
F1-score: 0.90


In [ ]:
model_3 = VGG(num_classes=10).to(device)
f_train_loss_list_3, f_metrics_3 = training_and_evaluating(model_3, train_loader, test_loader, 10)
f_metrics_3['Dataset']='FASHION MNIST'
f_metrics_3['Model name']='VGG'
metrics_f.append(f_metrics_3)

Epoch [1/10], Loss: 1.3334
Epoch [2/10], Loss: 0.5549
Epoch [3/10], Loss: 0.4125
Epoch [4/10], Loss: 0.3572
Epoch [5/10], Loss: 0.3219
Epoch [6/10], Loss: 0.3245
Epoch [7/10], Loss: 0.2730
Epoch [8/10], Loss: 0.2784
Epoch [9/10], Loss: 0.2627
Epoch [10/10], Loss: 0.2568

Accuracy: 90.74%
Precision: 0.92
Recall: 0.92
F1-score: 0.92


In [ ]:
model_4 = ResNet18(num_classes=10).to(device)
f_train_loss_list_4, f_metrics_4 = training_and_evaluating(model_4, train_loader, test_loader, 10)
f_metrics_4['Dataset']='FASHION MNIST'
f_metrics_4['Model name']='ResNet18'
metrics_f.append(f_metrics_4)

Epoch [1/10], Loss: 0.4311
Epoch [2/10], Loss: 0.3106
Epoch [3/10], Loss: 0.2703
Epoch [4/10], Loss: 0.2453
Epoch [5/10], Loss: 0.2299
Epoch [6/10], Loss: 0.2118
Epoch [7/10], Loss: 0.1986
Epoch [8/10], Loss: 0.1833
Epoch [9/10], Loss: 0.1693
Epoch [10/10], Loss: 0.1533

Accuracy: 92.05%
Precision: 0.93
Recall: 0.94
F1-score: 0.93


In [ ]:
model_5 = SENet18(num_classes=10).to(device)
f_train_loss_list_5, f_metrics_5 = training_and_evaluating(model_5, train_loader, test_loader, 10)
f_metrics_5['Dataset']='FASHION MNIST'
f_metrics_5['Model name']='SENet18'
metrics_f.append(f_metrics_5)

Epoch [1/10], Loss: 0.4080
Epoch [2/10], Loss: 0.2967
Epoch [3/10], Loss: 0.2585
Epoch [4/10], Loss: 0.2357
Epoch [5/10], Loss: 0.2142
Epoch [6/10], Loss: 0.1934
Epoch [7/10], Loss: 0.1781
Epoch [8/10], Loss: 0.1646
Epoch [9/10], Loss: 0.1491
Epoch [10/10], Loss: 0.1328

Accuracy: 91.71%
Precision: 0.93
Recall: 0.93
F1-score: 0.93


In [ ]:
model_6 = GoogleNet(num_classes=10).to(device)
f_train_loss_list_6, f_metrics_6 = training_and_evaluating(model_6, train_loader, test_loader, 10)
f_metrics_6['Dataset']='FASHION MNIST'
f_metrics_6['Model name']='GoogleNet'
metrics_f.append(f_metrics_6)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/10], Loss: 0.6461
Epoch [2/10], Loss: 0.3755
Epoch [3/10], Loss: 0.3464
Epoch [4/10], Loss: 0.3411
Epoch [5/10], Loss: 0.3385
Epoch [6/10], Loss: 0.3358
Epoch [7/10], Loss: 0.3597
Epoch [8/10], Loss: 0.3280
Epoch [9/10], Loss: 18169062271.1433
Epoch [10/10], Loss: 2.3034

Accuracy: 10.00%
Precision: 0.01
Recall: 0.10
F1-score: 0.02


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Creating CSV File**

In [ ]:
import pandas as pd
f_loss = {}
f_loss['LeNet5'] = f_train_loss_list_1
f_loss['AlexNet'] = f_train_loss_list_2
f_loss['VGG'] = f_train_loss_list_3
f_loss['ResNet18'] = f_train_loss_list_4
f_loss['SENet18'] = f_train_loss_list_5
f_loss['GoogleNet'] = f_train_loss_list_6

df_f_loss = pd.DataFrame(f_loss)
df_f_loss.to_csv("fashion_mnist_loss.csv",index=False)

In [ ]:
df_f_metrics = pd.DataFrame(metrics_f)
df_f_metrics.to_csv("fashion_mnist_metrics.csv",index=False)

**CIFAR-10 DATA**

In [11]:
#creating a list for evaluation metrics of all models for CIFAR-10 Dataset
metrics_c = []

In [12]:
#Data_loader for CIFAR-10 DATA
from torch.utils.data import DataLoader
train_loader = DataLoader(data['CIFAR-10'][0], batch_size=64, shuffle=True)
test_loader = DataLoader(data['CIFAR-10'][1], batch_size=1000, shuffle=False)

**Function call for each model**

In [13]:
#parameters of training_and_evaluating fumction --> model, train_loader, test_loader, num_epochs

model_1 = LeNet5(num_classes=10).to(device)
c_train_loss_list_1, c_metrics_1 = training_and_evaluating(model_1, train_loader, test_loader, 20)
c_metrics_1['Dataset']='CIFAR-10'
c_metrics_1['Model name']='LeNet5'
metrics_c.append(c_metrics_1)

Epoch [1/20], Loss: 1.6633
Epoch [2/20], Loss: 1.3550
Epoch [3/20], Loss: 1.2386
Epoch [4/20], Loss: 1.1604
Epoch [5/20], Loss: 1.0993
Epoch [6/20], Loss: 1.0486
Epoch [7/20], Loss: 1.0060
Epoch [8/20], Loss: 0.9651
Epoch [9/20], Loss: 0.9309
Epoch [10/20], Loss: 0.8931
Epoch [11/20], Loss: 0.8649
Epoch [12/20], Loss: 0.8423
Epoch [13/20], Loss: 0.8143
Epoch [14/20], Loss: 0.7915
Epoch [15/20], Loss: 0.7656
Epoch [16/20], Loss: 0.7477
Epoch [17/20], Loss: 0.7283
Epoch [18/20], Loss: 0.7052
Epoch [19/20], Loss: 0.6906
Epoch [20/20], Loss: 0.6733

Accuracy: 63.98%
Precision: 0.63
Recall: 0.63
F1-score: 0.63


In [14]:
model_2 = AlexNet(num_classes=10).to(device)
c_train_loss_list_2, c_metrics_2= training_and_evaluating(model_2, train_loader, test_loader, 20)
c_metrics_2['Dataset']='CIFAR-10'
c_metrics_2['Model name']='AlexNet'
metrics_c.append(c_metrics_2)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/20], Loss: 1.8664
Epoch [2/20], Loss: 1.5385
Epoch [3/20], Loss: 1.3848
Epoch [4/20], Loss: 1.2809
Epoch [5/20], Loss: 1.1963
Epoch [6/20], Loss: 1.1298
Epoch [7/20], Loss: 1.0704
Epoch [8/20], Loss: 1.0084
Epoch [9/20], Loss: 0.9589
Epoch [10/20], Loss: 0.9138
Epoch [11/20], Loss: 0.8679
Epoch [12/20], Loss: 0.8332
Epoch [13/20], Loss: 0.7835
Epoch [14/20], Loss: 0.7430
Epoch [15/20], Loss: 0.7023
Epoch [16/20], Loss: 0.6748
Epoch [17/20], Loss: 0.6274
Epoch [18/20], Loss: 0.5994
Epoch [19/20], Loss: 0.5791
Epoch [20/20], Loss: 0.5342

Accuracy: 61.75%
Precision: 0.60
Recall: 0.60
F1-score: 0.59


In [15]:
model_3 = VGG(num_classes=10).to(device)
c_train_loss_list_3, c_metrics_3 = training_and_evaluating(model_3, train_loader, test_loader, 20)
c_metrics_3['Dataset']='CIFAR-10'
c_metrics_3['Model name']='VGG'
metrics_c.append(c_metrics_3)

Epoch [1/20], Loss: 2.3035
Epoch [2/20], Loss: 2.3030
Epoch [3/20], Loss: 2.3030
Epoch [4/20], Loss: 2.3028
Epoch [5/20], Loss: 2.3028
Epoch [6/20], Loss: 2.3028
Epoch [7/20], Loss: 2.3028
Epoch [8/20], Loss: 2.3029
Epoch [9/20], Loss: 2.3027
Epoch [10/20], Loss: 2.3027
Epoch [11/20], Loss: 2.3027
Epoch [12/20], Loss: 2.3027
Epoch [13/20], Loss: 2.3027
Epoch [14/20], Loss: 2.3027
Epoch [15/20], Loss: 2.3027
Epoch [16/20], Loss: 2.3027
Epoch [17/20], Loss: 2.3027
Epoch [18/20], Loss: 2.3027
Epoch [19/20], Loss: 2.3028
Epoch [20/20], Loss: 2.3027

Accuracy: 10.00%
Precision: 0.01
Recall: 0.10
F1-score: 0.02


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
model_4 = ResNet18(num_classes=10).to(device)
c_train_loss_list_4, c_metrics_4= training_and_evaluating(model_4, train_loader, test_loader, 20)
c_metrics_4['Dataset']='CIFAR-10'
c_metrics_4['Model name']='ResNet18'
metrics_c.append(c_metrics_4)

Epoch [1/20], Loss: 1.3957
Epoch [2/20], Loss: 0.9884
Epoch [3/20], Loss: 0.8179
Epoch [4/20], Loss: 0.6950
Epoch [5/20], Loss: 0.6010
Epoch [6/20], Loss: 0.5088
Epoch [7/20], Loss: 0.4276
Epoch [8/20], Loss: 0.3528
Epoch [9/20], Loss: 0.2843
Epoch [10/20], Loss: 0.2251
Epoch [11/20], Loss: 0.1866
Epoch [12/20], Loss: 0.1548
Epoch [13/20], Loss: 0.1311
Epoch [14/20], Loss: 0.1121
Epoch [15/20], Loss: 0.0986
Epoch [16/20], Loss: 0.0953
Epoch [17/20], Loss: 0.1039
Epoch [18/20], Loss: 0.0719
Epoch [19/20], Loss: 0.0687
Epoch [20/20], Loss: 0.0703

Accuracy: 77.18%
Precision: 0.78
Recall: 0.78
F1-score: 0.78


In [17]:
model_5 = SENet18(num_classes=10).to(device)
c_train_loss_list_5, c_metrics_5 = training_and_evaluating(model_5, train_loader, test_loader, 20)
c_metrics_5['Dataset']='CIFAR-10'
c_metrics_5['Model name']='SENet18'
metrics_c.append(c_metrics_5)

Epoch [1/20], Loss: 1.3110
Epoch [2/20], Loss: 0.9305
Epoch [3/20], Loss: 0.7626
Epoch [4/20], Loss: 0.6413
Epoch [5/20], Loss: 0.5363
Epoch [6/20], Loss: 0.4418
Epoch [7/20], Loss: 0.3493
Epoch [8/20], Loss: 0.2812
Epoch [9/20], Loss: 0.2175
Epoch [10/20], Loss: 0.1781
Epoch [11/20], Loss: 0.1452
Epoch [12/20], Loss: 0.1276
Epoch [13/20], Loss: 0.1111
Epoch [14/20], Loss: 0.0924
Epoch [15/20], Loss: 0.0905
Epoch [16/20], Loss: 0.0840
Epoch [17/20], Loss: 0.0822
Epoch [18/20], Loss: 0.0737
Epoch [19/20], Loss: 0.0647
Epoch [20/20], Loss: 0.0617

Accuracy: 76.20%
Precision: 0.78
Recall: 0.78
F1-score: 0.78


In [18]:
model_6 = GoogleNet(num_classes=10).to(device)
c_train_loss_list_6, c_metrics_6 = training_and_evaluating(model_6, train_loader, test_loader, 20)
c_metrics_6['Dataset']='CIFAR-10'
c_metrics_6['Model name']='GoogleNet'
metrics_c.append(c_metrics_6)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/20], Loss: 1.6047
Epoch [2/20], Loss: 1.1880
Epoch [3/20], Loss: 1.0358
Epoch [4/20], Loss: 0.9572
Epoch [5/20], Loss: 0.9048
Epoch [6/20], Loss: 0.8734
Epoch [7/20], Loss: 0.8426
Epoch [8/20], Loss: 0.8305
Epoch [9/20], Loss: 0.8068
Epoch [10/20], Loss: 0.8076
Epoch [11/20], Loss: 0.8184
Epoch [12/20], Loss: 0.7802
Epoch [13/20], Loss: 0.7738
Epoch [14/20], Loss: 0.7793
Epoch [15/20], Loss: 0.8117
Epoch [16/20], Loss: 0.8034
Epoch [17/20], Loss: 0.7954
Epoch [18/20], Loss: 0.7959
Epoch [19/20], Loss: 0.7859
Epoch [20/20], Loss: 0.8345

Accuracy: 10.00%
Precision: 0.01
Recall: 0.10
F1-score: 0.02


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Creating CSV File**

In [19]:
import pandas as pd
c_loss = {}
c_loss['LeNet5'] = c_train_loss_list_1
c_loss['AlexNet'] = c_train_loss_list_2
c_loss['VGG'] = c_train_loss_list_3
c_loss['ResNet18'] = c_train_loss_list_4
c_loss['SENet18'] = c_train_loss_list_5
c_loss['GoogleNet'] = c_train_loss_list_6

df_c_loss = pd.DataFrame(c_loss)
df_c_loss.to_csv("cifar_10_loss.csv",index=False)

In [20]:
df_c_metrics = pd.DataFrame(metrics_c)
df_c_metrics.to_csv("cifar_10_metrics.csv",index=False)